# Linear Regression - Neural Network

## Data Prep

In [33]:
# Read data
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X, y = data.data, data.target

In [34]:
n_features = X.shape[1]
n_features

8

In [35]:
from sklearn.model_selection import train_test_split
# train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=0)

## PyTorch Dataset


In [36]:
# Pytorch Dataset
import torch
from torch.utils.data import Dataset, DataLoader

class IncomeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1) # or .reshape(-1,1).

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [37]:
# Pytorch
train_dataset = IncomeDataset(X=X_train, y=y_train)
test_dataset = IncomeDataset(X=X_test, y=y_test)

In [38]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

## Neural Network Model

In [39]:
# Neuaral Network Model
import torch.nn as nn
class IncomeModel(nn.Module):
    def __init__(self):
        super(IncomeModel, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(in_features=8, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1), # output for regression
        )

    def forward(self, x):
        return self.network(x)

## Training

In [40]:
model = IncomeModel()

In [41]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Loss
criterion = nn.MSELoss() # MSE for Linear Regression
epochs = 100


In [42]:
# training loop
for epoch in range(epochs):
    model.train()  # important: sets layers like Dropout, BatchNorm to training mode
    running_loss = 0
    for inputs, targets in train_loader:
        outputs = model(inputs)               # forward pass
        loss = criterion(outputs, targets)    # compute loss

        optimizer.zero_grad()  # clear gradients
        loss.backward()        # backpropagation
        optimizer.step()       # update weights
        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 10/100, Loss: 2.7024
Epoch 20/100, Loss: 1.1751
Epoch 30/100, Loss: 1.0409
Epoch 40/100, Loss: 0.8651
Epoch 50/100, Loss: 0.7493
Epoch 60/100, Loss: 0.7082
Epoch 70/100, Loss: 0.6695
Epoch 80/100, Loss: 0.6449
Epoch 90/100, Loss: 0.6263
Epoch 100/100, Loss: 0.6173
